<a href="https://colab.research.google.com/github/satani99/fastai_22_2/blob/main/Learner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/fastai/course22p2.git
%cd course22p2

Cloning into 'course22p2'...
remote: Enumerating objects: 1176, done.
remote: Counting objects: 100% (1176/1176), done.
remote: Compressing objects: 100% (480/480), done.
remote: Total 1176 (delta 712), reused 1117 (delta 694), pack-reused 0
Receiving objects: 100% (1176/1176), 99.81 MiB | 23.91 MiB/s, done.
Resolving deltas: 100% (712/712), done.
Updating files: 100% (138/138), done.
/content/course22p2


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.1 MB/s eta 0:00:00


In [3]:
import math, torch, matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from operator import attrgetter
from functools import partial
from copy import copy

from torch import optim
import torch.nn.functional as F

from miniai.conv import *

from fastprogress import progress_bar, master_bar

In [4]:
import matplotlib as mpl
import torchvision.transforms.functional as TF
from contextlib import contextmanager
from torch import nn, tensor
from datasets import load_dataset, load_dataset_builder
from miniai.conv import *
from miniai.datasets import *
import logging
from fastcore.test import test_close

In [5]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(42)
mpl.rcParams['image.cmap'] = 'gray'

In [6]:
logging.disable(logging.WARNING)

In [7]:
x, y = 'image', 'label'
name = 'fashion_mnist'
dsd = load_dataset(name)

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
@inplace
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [9]:
bs = 1024
tds = dsd.with_transform(transformi)

In [10]:
dls = DataLoaders.from_dd(tds, bs, num_workers=4)
dt = dls.train
xb, yb = next(iter(dt))
xb.shape, yb[:10]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


(torch.Size([1024, 784]), tensor([5, 7, 4, 7, 3, 8, 9, 5, 3, 1]))

In [11]:
class Learner:
  def __init__(self, model, dls, loss_func, lr, opt_func=optim.SGD): fc.store_attr()

  def one_batch(self):
    self.xb, self.yb = to_device(self.batch)
    self.preds = self.model(self.xb)
    self.loss = self.loss_func(self.preds, self.yb)
    if self.model.training:
      self.loss.backward()
      self.opt.step()
      self.opt.zero_grad()
    with torch.no_grad(): self.calc_stats()

  def calc_stats(self):
    acc = (self.preds.argmax(dim=1)==self.yb).float().sum()
    self.accs.append(acc)
    n = len(self.xb)
    self.losses.append(self.loss*n)
    self.ns.append(n)

  def one_epoch(self, train):
    self.model.training = train
    dl = self.dls.train if train else self.dls.valid
    for self.num, self.batch in enumerate(dl): self.one_batch()
    n = sum(self.ns)
    print(self.epoch, self.model.training, sum(self.losses).item()/n, sum(self.accs).item()/n)

  def fit(self, n_epochs):
    self.accs, self.losses, self.ns = [], [], []
    self.model.to(def_device)
    self.opt = self.opt_func(self.model.parameters(), self.lr)
    self.n_epochs = n_epochs
    for self.epoch in range(n_epochs):
      self.one_epoch(True)
      with torch.no_grad(): self.one_epoch(False)


In [12]:
m, nh = 28*28, 50
model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))

In [13]:
learn = Learner(model, dls, F.cross_entropy, lr=0.2)
learn.fit(1)

0 True 1.2019524739583334 0.5945166666666667
0 False 1.14890390625 0.6077285714285714


In [14]:
class CancelFitException(Exception): pass
class CancelBatchException(Exception): pass
class CancelEpochException(Exception): pass

In [15]:
class Callback(): order = 0

In [16]:
def run_cbs(cbs, method_nm, learn=None):
  for cb in sorted(cbs, key=attrgetter('order')):
    method = getattr(cb, method_nm, None)
    if method is not None: method(learn)

In [17]:
class CompletionCB(Callback):
  def before_fit(self, learn): self.count = 0
  def after_batch(self, learn): self.count += 1
  def after_fit(self, learn): print(f'Completed {self.count} batches')

In [18]:
cbs = [CompletionCB()]
run_cbs(cbs, 'before_fit')
run_cbs(cbs, 'after_batch')
run_cbs(cbs, 'after_fit')

Completed 1 batches


In [19]:
class Learner():
  def __init__(self, model, dls, loss_func, lr, cbs, opt_func=optim.SGD): fc.store_attr()

  def one_batch(self):
    self.preds = self.model(self.batch[0])
    self.loss = self.loss_func(self.preds, self.batch[1])
    if self.model.training:
      self.loss.backward()
      self.opt.step()
      self.opt.zero_grad()

  def one_epoch(self, train):
    self.model.train(train)
    self.dl = self.dls.train if train else self.dls.valid
    try:
      self.callback('before_epoch')
      for self.iter, self.batch in enumerate(self.dl):
        try:
          self.callback('before_batch')
          self.one_batch()
          self.callback('after_batch')
        except CancelBatchException: pass
      self.callback('after_epoch')
    except CancelEpochException: pass

  def fit(self, n_epochs):
    self.n_epochs = n_epochs
    self.epochs = range(n_epochs)
    self.opt = self.opt_func(self.model.parameters(), self.lr)
    try:
      self.callback('before_fit')
      for self.epoch in self.epochs:
        self.one_epoch(True)
        self.one_epoch(False)
      self.callback('after_fit')
    except CancelFitException: pass

  def callback(self, method_nm): run_cbs(self.cbs, method_nm, self)

In [20]:
m, nh = 28*28, 50
def get_model(): return nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))

In [21]:
model = get_model()
learn = Learner(model, dls, F.cross_entropy, lr=0.2, cbs=[CompletionCB()])
learn.fit(1)

Completed 64 batches


In [22]:
class SingleBatchCB(Callback):
  order = 1
  def after_batch(self, learn): raise CancelFitException()

In [23]:
learn = Learner(get_model(), dls, F.cross_entropy, lr=0.2, cbs=[CompletionCB(), SingleBatchCB()])
learn.fit(1)

In [24]:
class Metric:
  def __init__(self): self.reset()
  def reset(self): self.vals, self.ns = [], []
  def add(self, inp, targ=None, n=1):
    self.last = self.calc(inp, targ)
    self.vals.append(self.last)
    self.ns.append(n)
  @property
  def value(self):
    ns = tensor(self.ns)
    return (tensor(self.vals)*ns).sum()/ns.sum()
  def calc(self, inps, targs): return inps

In [25]:
class Accuracy(Metric):
  def calc(self, inps, targs): return (inps==targs).float().mean()

In [26]:
acc = Accuracy()
acc.add(tensor([0, 1, 2, 0, 1, 2]), tensor([0, 1, 1, 2, 1, 0]))
acc.add(tensor([1, 1, 2, 0, 1]), tensor([0, 1, 1, 2, 1]))
acc.value

tensor(0.45)

In [27]:
loss = Metric()
loss.add(0.6, n=32)
loss.add(0.9, n=2)
loss.value, round((0.6*32 + 0.9*2)/(32+2), 2)

(tensor(0.62), 0.62)

In [28]:
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.4 MB/s eta 0:00:00


In [29]:
from torcheval.metrics import MulticlassAccuracy, Mean

In [30]:
metric = MulticlassAccuracy()
metric.update(tensor([0, 2, 1, 3]), tensor([0, 1, 2, 3]))
metric.compute()

tensor(0.50)

In [31]:
metric.reset()
metric.compute()

tensor(nan)

In [32]:
def to_cpu(x):
  if isinstance(x, Mapping): return {k:to_cpu(v) for k,v in x.items()}
  if isinstance(x, list): return [to_cpu(o) for o in x]
  if isinstance(x, tuple): return tuple(to_cpu(list(x)))
  res = x.detach().cpu()
  return res.float() if res.dtype==torch.float16 else res

In [34]:
class MetricsCB(Callback):
  def __init__(self, *ms, **metrics):
    for o in ms: metrics[type(o).__name__] = o
    self.metrics = metrics
    self.all_metrics = copy(metrics)
    self.all_metrics['loss'] = self.loss = Mean()

  def _log(self, d): print(d)
  def before_fit(self, learn): learn.metrics = self
  def before_epoch(self, learn): [o.reset() for o in self.all_metrics.values()]

  def after_epoch(self, learn):
    log = {k:f'{v.compute():.3f}' for k,v in self.all_metrics.items()}
    log['epoch'] = learn.epoch
    log['train'] = 'train' if learn.model.training else 'eval'
    self._log(log)

  def after_batch(self, learn):
    x, y, *_ = to_cpu(learn.batch)
    for m in self.metrics.values(): m.update(to_cpu(learn.preds), y)
    self.loss.update(to_cpu(learn.loss), weight=len(x))

In [37]:
class DeviceCB(Callback):
  def __init__(self, device=def_device): fc.store_attr()
  def before_fit(self, learn):
    if hasattr(learn.model, 'to'): learn.model.to(self.device)
  def before_batch(self, learn): learn.batch = to_device(learn.batch, device=self.device)

In [40]:
model = get_model()
metrics = MetricsCB(accuracy=MulticlassAccuracy())
learn = Learner(model, dls, F.cross_entropy, lr=0.2, cbs=[DeviceCB(), metrics])
learn.fit(1)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


{'accuracy': '0.595', 'loss': '1.175', 'epoch': 0, 'train': 'train'}
{'accuracy': '0.717', 'loss': '0.768', 'epoch': 0, 'train': 'eval'}


In [ ]:
class Learner():
  def __init__(self, model, dls=(0,), loss_func=F.mse_loss, lr=0.1, cbs=None, opt_func=optim.SGD):
    cbs = fc.L(cbs)
    fc.store_attr()

  @contextmanager
  def cb_ctx(self, nm):
    try:
      self.callback(f'before_{nm}')
      yield
      self.callback(f'after_{nm}')
    except globals()[f'Cancel{nm.title()}Exception']: pass
    finally: self.callback(f'cleanup_{nm}')

  def one_epoch(self, train):
    self.model.train(train)
    self.dl = self.dls.train if train else self.dls.valid
    with self.cb_ctx('epoch'):
      for self.iter, self.batch in enumerate(self.dl):
        with self.cb_ctx('batch'):
          self.predict()
          self.get_loss()
          if self.training:
            self.backward()
            self.step()
            self.zero_grad()


  def fit(self, n_epochs=1, train=True, valid=True, cbs=None, lr=None):
    cbs = fc.L(cbs)
